In [ ]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import pylab as pyl
import time
import cvxpy as cp
from numpy import linalg as Lin
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import style
from sklearn import datasets

import warnings

warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def distmat(x,y):
    
    return np.sum(x**2,0)[:,None] + np.sum(y**2,0)[None,:] - 2*x.transpose().dot(y)


In [ ]:
def plotp(plt, x, col, scale=200, edgecolors="k"):
  return plt.scatter(x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2)

# Helper Functions

In [ ]:
normalize = lambda a: a/np.sum(a)

In [ ]:
def GetP(u,K,v):
    return u*K*(v.T)

# Annulus vs Square

## Trying for different dimensions n x n

#### Generate Data

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def randomsampledata(N):
  x=[]
  y=[]
  N=np.sort(N)
  for i in range(len(N)):
    x.append(np.random.rand(2,N[i])-0.5)
    theta=2*np.pi*np.random.rand(1,N[i])
    r =0.8+0.2*np.random.rand(1,N[i])
    y.append(np.vstack((np.cos(theta)*r,np.sin(theta)*r)))
  
  return x,y,N

In [ ]:
N=[200,400,600,800,1000]
x,y,N=randomsampledata(N)

### Linear Optimization

In [ ]:
import computational_OT
#from linear import LinearOptimize

In [ ]:
times_linearOpt=[]
LinearP=[]
for i in range(len(N)):
  xi,yi=x[i],y[i]
  #Cost matrix
  C = distmat(xi,yi)

  # a and b
  a = normalize(np.ones(N[i]))
  a=a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[i]))
  b=b.reshape(b.shape[0],-1)



  plt.figure(figsize=(5,5))

  plotp(plt,xi, col='b')
  plotp(plt,yi, col='r')

  plt.axis("off")
  plt.xlim(np.min(yi[0,:])-.1,np.max(yi[0,:])+.1)
  plt.ylim(np.min(yi[1,:])-.1,np.max(yi[1,:])+.1)

  plt.show()




  print("Doing for ",N[i])
  print( " |- Iterating")
  ### Linear Optimization
  start=time.time()
  Optimizer = computational_OT.LinearOptimize(N[i],N[i],a,b,C)
  print( " |- Computing P")
  print( "" )
  P_linear=Optimizer.solve()
  LinearP.append(P_linear.value)
  end=time.time()
  times_linearOpt.append(end-start)

### Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
Pmatrix_dist_linVSsinkhorn=[]
for i in range(len(N)):

  
  xi,yi=x[i],y[i]
  #Cost matrix
  C = distmat(xi,yi)
  
  # a and b
  a = normalize(np.ones(N[i]))
  a=a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[i]))
  b=b.reshape(b.shape[0],-1)
  
  #Epsilon
  epsilon = .06

  #Kernel
  K=np.exp(-C/epsilon)


  print("Doing for ",N[i])
  print( " |- Iterating")

  #Inflating
  u=a
  v=b

  start=time.time()
  Optimizer=computational_OT.Sinkhorn(N[i],K,a,b,u,v,epsilon)
  out=Optimizer._update()
  results_Sinkhorn.append(out)
  end=time.time()
  times_Sinkhorn.append(end-start)
  print( " |- Computing P")
  print( "" )
  SinkhornP.append(GetP(out[0],K,out[1]))
  

##### Convergence plot for Sinkhorn

In [ ]:
plt.figure(figsize = (20,7))

plt.subplot(2,1,1),
plt.title("$||P1 -a||_1$")
for result in results_Sinkhorn:
  plt.plot( np.asarray(result[2]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")

plt.subplot(2,1,2)
plt.title("$||P^T 1 -b||_1$")
for result in results_Sinkhorn:
  plt.plot( np.asarray(result[3]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

##### Objective Function

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Objective Function")


for result in results_Sinkhorn:
  plt.plot(np.asarray(result[4]).flatten(), linewidth = 2)
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

### Gradient Ascent

In [ ]:
learning_rate=0.005

GradientAscentP=[]
results_Gradient_ascent=[]
times_Gradient_ascent=[]
Pmatrix_dist_linVSgradientascent=[]
Pmatrix_dist_sinkhornVSgradientascent=[]

# Gradient ascent
print("Gradient ascent....")
results_Gradient_ascent=[]
times_Gradient_ascent=[]
for i in range(len(N)):

  xi,yi=x[i],y[i]
  #Cost matrix
  C = distmat(xi,yi)

  # a and b
  a = normalize(np.ones(N[i]))
  a=a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[i]))
  b=b.reshape(b.shape[0],-1)

  #Epsilon
  epsilon = .06

  #Kernel
  K=np.exp(-C/epsilon)
  
  f,g=a,b
  


  print("Doing for ",N[i])
  print( " |- Iterating")
  start=time.time()
  Optimizer=computational_OT.Gradient_Ascent(K,a,b,f,g,epsilon,learning_rate)
  out=Optimizer._update()
  results_Gradient_ascent.append(out)
  end=time.time()
  times_Gradient_ascent.append(end-start)
  print( " |- Computing P")
  print( "" )
  GradientAscentP.append(GetP(np.exp(out[0]/epsilon),K,np.exp(out[1]/epsilon)))
  


##### Convergence plot for Gradient Ascent

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("$||P1 -a||_1$")

for result in results_Gradient_ascent:
  plt.plot(np.asarray(result[2]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.subplot(2,1,2)

plt.title("$||P^T 1 -b||_1$")
for result in results_Gradient_ascent:
  plt.plot(np.asarray(result[3]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

##### Objective Function

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Objective Function")


for result in results_Gradient_ascent:
  plt.plot(np.asarray(result[4]).flatten(), linewidth = 2)
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

### Line Search

In [ ]:
rho=0.95
rho_inc=1.5
c1=0.05
c2=0.9
initial_alpha=1

LineSearchP=[]
results_LineSearch=[]
times_LineSearch=[]
Pmatrix_dist_linVSLineSearchP=[]
Pmatrix_dist_sinkhornVSLineSearchP=[]

Pmatrix_dist_GradientAscentVSLineSearchP=[]


# Line Search
print("Line Search....")
for i in range(len(N)):

      xi,yi=x[i],y[i]
      #Cost matrix
      C = distmat(xi,yi)

      # a and b
      a = normalize(np.ones(N[i]))
      a=a.reshape(a.shape[0],-1)
      b = normalize(np.ones(N[i]))
      b=b.reshape(b.shape[0],-1)

      #Epsilon
      epsilon = .06

      #Kernel
      K=np.exp(-C/epsilon)


      f,g=a,b
              

      print("Doing for ",N[i])
      print( " |- Iterating")
      start=time.time()
      Optimizer=computational_OT.LineSearch(K,a,b,f,g,epsilon,rho,rho_inc,c1,c2,initial_alpha)
      out=Optimizer._update()
      results_LineSearch.append(out)
      end=time.time()
      times_LineSearch.append(end-start)
      print( " |- Computing P")
      print( "" )
      LineSearchP.append(GetP(np.exp(out[0]/epsilon),K,np.exp(out[1]/epsilon)))

     

##### Convergence plot for Line Search

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.subplot(2,1,1),
plt.title("$||P1 -a||_1$")

for result in results_LineSearch:
  plt.plot(np.asarray(result[2]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.subplot(2,1,2)

plt.title("$||P^T 1 -b||_1$")
for result in results_LineSearch:
  plt.plot(np.asarray(result[3]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

##### Objective Function

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Objective Function")


for result in results_LineSearch:
  plt.plot(np.asarray(result[4]), linewidth = 2)
#plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

##### Alpha plot

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Alpha")

for result in results_LineSearch:
  plt.plot(np.asarray(result[5]), linewidth = 2)
#plt.yscale( 'log')

plt.legend([str(i) for i in N],loc="upper right")
plt.show()

### L_BFGS_B

In [ ]:
BFGSP=[]
results_BFGS=[]

times_BFGS=[]
Pmatrix_dist_linVSBFGSP=[]
Pmatrix_dist_sinkhornVSBFGSP=[]
Pmatrix_dist_GradientAscentVSBFGSP=[]

# BFGS
print("BFGS....")

for i in range(len(N)):

      xi,yi=x[i],y[i]
      #Cost matrix
      C = distmat(xi,yi)
      # a and b
      a = normalize(np.ones(N[i]))
      a=a.reshape(a.shape[0],-1)
      b = normalize(np.ones(N[i]))
      b=b.reshape(b.shape[0],-1)

      #Epsilon
      epsilon = .06

      #Kernel
      K=np.exp(-C/epsilon)


      f,g=a,b


      print("Doing for ",N[i])
      print( " |- Iterating")
      start=time.time()
      Optimizer=computational_OT.L_BFGS_B(K,a,b,f,g,epsilon)
      out=Optimizer._update()
      results_BFGS.append(out)
      end=time.time()
      times_BFGS.append(end-start)
      print( " |- Computing P")
      print( "" )
      BFGSP.append(GetP(np.exp(out[0]/epsilon),K,np.exp(out[1]/epsilon)))

     

##### Convergence plot for L-BFGS-B

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.subplot(2,1,1),
plt.title("$||P1 -a||_1$")

for result in results_BFGS:
  plt.plot(np.asarray(result[2]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.subplot(2,1,2)

plt.title("$||P^T 1 -b||_1$")
for result in results_BFGS:
  plt.plot(np.asarray(result[3]), linewidth = 2)
plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

##### Objective Function

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Objective Function")


for result in results_BFGS:
  plt.plot(np.asarray(result[4]), linewidth = 2)
#plt.yscale( 'log')
plt.legend([str(i) for i in N],loc="upper right")
plt.show()

#### Time Plot

In [ ]:
plt.figure(figsize = (20,7))
plt.title("Time Plot")
plt.plot(list(range(1,len(times_linearOpt)+1)),times_linearOpt, '-rD' )
plt.plot(list(range(1,len(times_Sinkhorn)+1)),times_Sinkhorn, '-yD' )
plt.plot(list(range(1,len(times_Gradient_ascent)+1)),times_Gradient_ascent, '-bD' )
plt.plot(list(range(1,len(times_LineSearch)+1)),times_LineSearch, '-kD' )

plt.plot(list(range(1,len(times_BFGS)+1)),times_BFGS, '-gD' )
plt.yscale( 'log')
plt.ylabel("Time in sec, logscale")
plt.xlabel("Dimensions of samples drawn")
plt.xticks(list(range(1,len(times_BFGS)+1)),[str(i) for i in N])
plt.legend(["Linear Optimization","Sinkhorn","Gradient Ascent","Line Search","L-BFGS-B"],loc="upper left")
plt.show()